<a href="https://colab.research.google.com/github/Heity94/WalkieLookie/blob/main/notebooks/PH_Master_RoutingAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install requirements { display-mode: "form" }
!pip install osmnx -q
!pip install geopy -q

     |████████████████████████████████| 95 kB 1.4 MB/s 
     |████████████████████████████████| 11.2 MB 48.4 MB/s 
     |████████████████████████████████| 6.3 MB 8.1 MB/s 
     |████████████████████████████████| 62 kB 287 kB/s 
     |████████████████████████████████| 1.0 MB 60.3 MB/s 
     |████████████████████████████████| 1.0 MB 47.5 MB/s 
     |████████████████████████████████| 16.7 MB 272 kB/s 
     |████████████████████████████████| 944 kB 48.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
#@title Imports
import pickle
import folium
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
ox.config(log_console=True, use_cache=True)
from geopy.geocoders import Nominatim
#from networkx.algorithms import approximation as approx # Greedy salesman algorithms
import sklearn
from ast import literal_eval

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
# Link to common google drive folder (you have to add a link from the foder to your drive first)
#data_path = "/content/drive/MyDrive/BPI Lap WalkieLookie/Data/"
data_path = "/content/drive/MyDrive/WalkieLookie/BPI Lap WalkieLookie/Data" #Roger
#data_path="/home/valentin_werner/Philipps_Playground/WalkieLookie/data/" #Philipp Zeno

# Create own Routing algorithm

### Load data

Our algorithm needs two datasets:


1.   Dataframe of potenital places (nodes) of interest (e.g. all parks in Berlin)
2.   Street data from Berlin (graph data)



In [51]:
#Load DataFrame with nodes of interest (NOI)
places_noi = pd.read_csv(data_path+"/parks_bln_complete_clean.csv", index_col=0, converters={'nodes_on_graph': literal_eval})

In [52]:
places_noi.head()

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough
0,4413796,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259,1392678402,10707,Charlottenburg-Wilmersdorf
1,4440110,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11,-1,12305,Tempelhof-Schöneberg
2,4582243,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166,559228986,10713,Charlottenburg-Wilmersdorf
3,4582244,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103,-1,10713,Charlottenburg-Wilmersdorf
4,4592806,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497,1355566306,10557,Mitte


In [53]:
#load street data from berlin
with open(data_path+'/graph_berlin.obj', 'rb') as fp:
  street_graph = pickle.load(fp)

# **Roger Data Manipulation**

## **Add all_center_node column**

In [54]:
type(places_noi["nodes_on_graph"])

pandas.core.series.Series

In [55]:
def all_center_node(street_graph, places_nodes_on_map):
  '''Calculate center node of each place of interest, if no center found return string'''
  try: 
    return nx.center(street_graph.subgraph(places_nodes_on_map))#[0] #ALL CENTER NODES
  except:
    return "No center found"

In [56]:
#street_nodes = set(street_graph.nodes())
# Get center node for each park
places_noi["all_center_node"] = places_noi.nodes_on_graph.apply(lambda x: all_center_node(street_graph, x))

# filter out parks without center (for now)
#places_noi = places_noi[places_noi.all_center_node!="No center found"].copy()

# Select center park nodes and store as list for route planning
nodes_to_visit = places_noi.loc[:,"all_center_node"].to_list()

## **Add secondary_nodes**

In [64]:
#places_noi.to_csv(data_path+"/RogerDF.csv")
places_noi = pd.read_csv(data_path+"/RogerDF.csv", index_col=0, converters={'nodes_on_graph': literal_eval})

In [65]:
places_noi.head()

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough,all_center_node
0,4413796,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259,1392678402,10707,Charlottenburg-Wilmersdorf,"[1392678402, 260939035, 666379740, 279150573]"
1,4440110,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11,-1,12305,Tempelhof-Schöneberg,No center found
2,4582243,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166,559228986,10713,Charlottenburg-Wilmersdorf,"[559228986, 105388343]"
3,4582244,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103,-1,10713,Charlottenburg-Wilmersdorf,No center found
4,4592806,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497,1355566306,10557,Mitte,[1355566306]


In [69]:
tst = places_noi.iloc[[1]]
np.random.choice(tst.nodes_on_graph.values[0], size=tst.number_nodes_on_graph.values[0]//100, replace=False)

array([], dtype=int64)

In [ ]:
places_noi["number_nodes_on_graph"]

In [92]:
#number_nodes_on_graph to integer
#places_noi["number_nodes_on_graph"].astype("int")

secondary_nodes = []
np.random.seed(0)

for park in range(0,places_noi.shape[0]):

  #run through all rows
  park_pick = places_noi.iloc[[park]]

  #pick random nodes depending on number_nodes_on_graph
  node_pick = np.random.choice(park_pick.nodes_on_graph.values[0], size=park_pick.number_nodes_on_graph.values[0]//100, replace=False)

  secondary_nodes.append(node_pick)

In [95]:
places_noi["secondary_nodes"] = secondary_nodes
places_noi.head()

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough,all_center_node,secondary_nodes
0,4413796,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259,1392678402,10707,Charlottenburg-Wilmersdorf,"[1392678402, 260939035, 666379740, 279150573]","[9400968198, 9400968113]"
1,4440110,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11,-1,12305,Tempelhof-Schöneberg,No center found,[]
2,4582243,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166,559228986,10713,Charlottenburg-Wilmersdorf,"[559228986, 105388343]",[105388361]
3,4582244,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103,-1,10713,Charlottenburg-Wilmersdorf,No center found,[105388407]
4,4592806,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497,1355566306,10557,Mitte,[1355566306],"[1358441536, 1355566302, 1355566422, 1357158474]"


## **Add center_node when center_node == -1**

In [97]:
#places_noi.to_csv(data_path+"/RogerDFwithSecondaryNodes.csv")
places_noi = pd.read_csv(data_path+"/RogerDFwithSecondaryNodes.csv", index_col=0, converters={'nodes_on_graph': literal_eval})

In [109]:
places_noi.iloc[[park]]

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough,all_center_node,secondary_nodes
0,4413796,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259,1392678402,10707,Charlottenburg-Wilmersdorf,"[1392678402, 260939035, 666379740, 279150573]",[9400968198 9400968113]


In [120]:
np.random.seed(0)

for park in range(0,places_noi.shape[0]):
  park_pick = places_noi.iloc[[park]]
  if park_pick.center_node.values == [-1]:
    #pick random nodes depending on number_nodes_on_graph
    node_pick_nc = np.random.choice(park_pick.nodes_on_graph.values[0], size=1)
    places_noi.iloc[park, 4]=node_pick_nc

places_noi.head(10)

,osmid,name,nodes_on_graph,number_nodes_on_graph,center_node,zip,borough,all_center_node,secondary_nodes
0,4413796,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259,1392678402,10707,Charlottenburg-Wilmersdorf,"[1392678402, 260939035, 666379740, 279150573]",[9400968198 9400968113]
1,4440110,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11,3740400534,12305,Tempelhof-Schöneberg,No center found,[]
2,4582243,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166,559228986,10713,Charlottenburg-Wilmersdorf,"[559228986, 105388343]",[105388361]
3,4582244,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103,105388414,10713,Charlottenburg-Wilmersdorf,No center found,[105388407]
4,4592806,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497,1355566306,10557,Mitte,[1355566306],[1358441536 1355566302 1355566422 1357158474]
5,4592869,Ottopark,"[325624896, 4933972629, 2268327536, 325626011,...",89,2268327537,10555,Mitte,[2268327537],[]
6,4603319,Volkspark Humboldthain,"[29493750, 1392494975, 29493751, 1392494943, 2...",968,1392494929,13357,Mitte,"[1392494929, 307681045, 307681046, 307681049, ...",[9844946856 3941693915 6184062234 9844946863 ...
7,4611881,Mendelssohn-Bartholdy-Park,"[1870173832, 7823298114, 7823298113, 782329811...",66,1870173798,10785,Friedrichshain-Kreuzberg,[1870173798],[]
8,4638994,Viktoriapark,"[241490614, 241490617, 241490606, 401579047, 9...",971,243337807,10965,Friedrichshain-Kreuzberg,"[243337807, 9794667282, 241490740, 241490801, ...",[9574968771 386862881 1977632165 335588952 9...
9,4675900,Carl-von-Ossietzky-Park,"[335324727, 2272451597, 335324728, 335324731, ...",105,1393477178,10559,Mitte,"[1393477178, 1393477223, 1393477239, 328419452...",[1393477188]


## **Save final df**

In [121]:
places_noi.to_csv(data_path+"/RogerFinalDF.csv")
#places_noi = pd.read_csv(data_path+"/RogerDFwithSecondaryNodes.csv", index_col=0, converters={'nodes_on_graph': literal_eval})